# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
# Helium at the center of the box
atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_LDA(lattice, atoms, positions; symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis; tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770429213971                   -0.52    9.0         
  2   -2.771692852633       -2.90       -1.32    1.0    181ms
  3   -2.771714355362       -4.67       -2.45    1.0    136ms
  4   -2.771714642263       -6.54       -3.15    1.0    169ms
  5   -2.771714714981       -7.14       -4.24    2.0    180ms
  6   -2.771714715154       -9.76       -4.55    1.0    140ms
  7   -2.771714715248      -10.03       -5.87    1.0    152ms
  8   -2.771714715250      -11.87       -6.74    2.0    164ms
  9   -2.771714715250   +  -14.21       -6.59    1.0    172ms
 10   -2.771714715250      -14.21       -8.12    1.0    154ms


-0.0001345724175514104

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_LDA(lattice, atoms, positions;
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770419648081                   -0.53    9.0         
  2   -2.771773255523       -2.87       -1.31    1.0    169ms
  3   -2.771801836074       -4.54       -2.58    1.0    160ms
  4   -2.771802048716       -6.67       -3.58    1.0    199ms
  5   -2.771802072727       -7.62       -3.93    2.0    158ms
  6   -2.771802074411       -8.77       -4.97    1.0    142ms
  7   -2.771802074473      -10.21       -5.32    2.0    184ms
  8   -2.771802074476      -11.57       -6.06    1.0    147ms
  9   -2.771802074476      -12.71       -6.37    2.0    183ms
 10   -2.771802074476      -13.72       -7.54    1.0    155ms
 11   -2.771802074476   +  -14.05       -7.94    2.0    182ms
 12   -2.771802074476   +  -14.21       -8.70    1.0    157ms


0.01761222108142084

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.0001345724175514104
Displaced dipole:  0.01761222108142084
Polarizability :   1.7746793498972249


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve in iter 1; step 1: normres = 2.493920830151e-01
[ Info: GMRES linsolve in iter 1; step 2: normres = 3.766551437164e-03
[ Info: GMRES linsolve in iter 1; step 3: normres = 2.852764826764e-04
[ Info: GMRES linsolve in iter 1; step 4: normres = 4.694602748364e-06
[ Info: GMRES linsolve in iter 1; step 5: normres = 1.088787955485e-08
[ Info: GMRES linsolve in iter 1; step 6: normres = 6.287115072455e-11
[ Info: GMRES linsolve in iter 1; step 7: normres = 1.101891213613e-12
[ Info: GMRES linsolve in iter 1; finished at step 7: normres = 1.101891213613e-12
[ Info: GMRES linsolve in iter 2; step 1: normres = 7.148288954286e-11
[ Info: GMRES linsolve in iter 2; step 2: normres = 1.304587274393e-11
[ Info: GMRES linsolve in iter 2; step 3: normres = 2.548555519253e-13
[ Info: GMRES linsolve in iter 2; finished at step 3: normres = 2.548555519253e-13
┌ Info: GMRES linsolve converged at iteration 2, step 3:
│ *  norm of residual = 2.548961225334455e-13
└ *  number of operat

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.